In [1]:
%%writefile main.py
import os
import json
import uvicorn
from fastapi import Request, FastAPI, Response
from google.cloud import storage

app = FastAPI(title = "S2DR3")

AIP_HEALTH_ROUTE = os.environ.get("AIP_HEALTH_ROUTE", "/health")
AIP_PREDICT_ROUTE = os.environ.get("AIP_PREDICT_ROUTE", "/predict")

def upload_to_bucket(blob_name, path_to_file, bucket_name):
    storage_client = storage.Client.from_service_account_json(
        'credentials.json'
    )
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(path_to_file)

    return blob.public_url

def write_content_to_file(file_name, str_content):
    with open(file_name, "w") as file:
        file.write(str_content)

@app.get(AIP_HEALTH_ROUTE, status_code = 200)
async def health():
    return {
        "response" : "ok"
    }

@app.post(AIP_PREDICT_ROUTE)
async def predict(request: Request):
    body = await request.json()
    write_content_to_file("data.txt", json.dumps(body))
    uploaded_url = upload_to_bucket("output/data.txt", "data.txt", "s2dr3-202312")
    return uploaded_url
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port = 8080)

Writing main.py


In [2]:
!gsutil cp gs://s2dr3-202312/credentials.json .

Writing credentials.json


In [10]:
%%writefile Dockerfile
FROM ubuntu:22.04
WORKDIR .
COPY main.py ./main.py
COPY credentials.json ./credentials.json
COPY s2dr3-20231220.1-cp310-cp310-linux_x86_64.whl ./s2dr3-20231220.1-cp310-cp310-linux_x86_64.whl

RUN apt-get update \
    && apt-get install -y python3-pip \
    && apt-get install -y python3-dev \
    && apt-get install -y gdal-bin \
    && apt-get install -y libgdal-dev \
    && apt-get install -y g++ \
    && export CPLUS_INCLUDE_PATH=/usr/include/gdal \
    && export C_INCLUDE_PATH=/usr/include/gdal

RUN pip install s2dr3-20231220.1-cp310-cp310-linux_x86_64.whl

RUN pip install uvicorn fastapi
RUN pip install --upgrade google-cloud-storage

RUN pip install s2dr3-20231220.1-cp310-cp310-linux_x86_64.whl
RUN gdalinfo --version
RUN python3 -c "from osgeo import gdal; print(gdal.__version__)"

EXPOSE 8080
ENTRYPOINT [ "python3", "main.py" ]


Overwriting Dockerfile


In [5]:
!gsutil cp gs://s2dr3-202312/s2dr3-20231220.1-cp310-cp310-linux_x86_64.whl .

Copying gs://s2dr3-202312/s2dr3-20231220.1-cp310-cp310-linux_x86_64.whl...
- [1 files][  3.4 MiB/  3.4 MiB]                                                
Operation completed over 1 objects/3.4 MiB.                                      


In [6]:
!ls

cloudbuild.yaml   Dockerfile  s2dr3-20231220.1-cp310-cp310-linux_x86_64.whl
credentials.json  main.py


In [7]:
%%writefile cloudbuild.yaml
steps:
# Build the container image
- name: 'gcr.io/cloud-builders/docker'
  args: ['build', '-t', 'gcr.io/s2dr3-202312/test-fast-api', '.']
# Push the container image to Container Registry
- name: 'gcr.io/cloud-builders/docker'
  args: ['push', 'gcr.io/s2dr3-202312/test-fast-api']

images:
- gcr.io/s2dr3-202312/test-fast-api

Overwriting cloudbuild.yaml


In [8]:
!gcloud config set project s2dr3-202312

Updated property [core/project].


In [9]:
!gcloud builds submit --config cloudbuild.yaml

Creating temporary tarball archive of 24 file(s) totalling 5.0 MiB before compression.
Uploading tarball of [.] to [gs://s2dr3-202312_cloudbuild/source/1704824508.691137-0eb488195cea454298860634e8ae8bf7.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/s2dr3-202312/locations/global/builds/05418450-9d15-4489-b4ff-30ae698a860e].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/05418450-9d15-4489-b4ff-30ae698a860e?project=862134799361 ].
 REMOTE BUILD OUTPUT
starting build "05418450-9d15-4489-b4ff-30ae698a860e"

FETCHSOURCE
Fetching storage object: gs://s2dr3-202312_cloudbuild/source/1704824508.691137-0eb488195cea454298860634e8ae8bf7.tgz#1704824509587587
Copying gs://s2dr3-202312_cloudbuild/source/1704824508.691137-0eb488195cea454298860634e8ae8bf7.tgz#1704824509587587...
/ [1 files][  3.5 MiB/  3.5 MiB]                                                
Operation completed over 1 objects/3.5 MiB.
BUILD
Starting Step #0
Step #0: Already have image (with di